In [ ]:
# !pip3 install dnspython
# !pip3 install tqdm

In [ ]:
from dns.resolver import query, NXDOMAIN, NoNameservers, Timeout, NoAnswer
from dns.name import from_unicode

def r(host):
    try:
        host_record = query(from_unicode(host), "A")
        if len(host_record) > 0:
            return "OK"

    except NXDOMAIN:
        return NXDOMAIN.__name__
    except NoNameservers:
        return NoNameservers.__name__
    except Timeout:
        return Timeout.__name__
    except NameError:
        return NameError.__name__
    except NoAnswer:
        return NoAnswer.__name__
    except Exception as e:
        return e.__class__.__name__ + ": " + str(e)

In [ ]:
import pandas as pd
df = pd.read_csv('03_ml_dom.csv', sep='\t')
df = df[:150000]
df.head()

In [ ]:
import concurrent
from tqdm import tqdm_notebook as tqdm

with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:

    future_to_url = {executor.submit(r, df.domain[ind]): ind for ind in range(len(df))}
    for future in concurrent.futures.as_completed(future_to_url):
        ind = future_to_url[future]
        df.loc[ind, "dns_ans"] = future.result()